AdventureWorks Database

In [7]:

import pyodbc
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns


# Server connection to MS SQL:
import pyodbc
connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=DESKTOP-PG968NO;DATABASE=AdventureWorks2016;UID=sa;PWD=Naples01')
cursor=connection.cursor()

In [6]:
df = pd.read_sql_query("""
SELECT BusinessEntityID
      ,Name
      ,AnnualSales
      ,AnnualRevenue
      ,BankName
      ,BusinessType
      ,YearOpened
      ,Specialty
      ,SquareFeet
      ,Brands
      ,Internet
      ,NumberEmployees
  FROM AdventureWorks2019.Sales.vStoreWithDemographics;
""", connection)
df.head()

,BusinessEntityID,Name,AnnualSales,AnnualRevenue,BankName,BusinessType,YearOpened,Specialty,SquareFeet,Brands,Internet,NumberEmployees
0,292,Next-Door Bike Store,800000.0,80000.0,United Security,BM,1996,Mountain,21000,2,ISDN,13
1,294,Professional Sales and Service,800000.0,80000.0,International Bank,BM,1991,Touring,18000,4+,T1,14
2,296,Riders Company,800000.0,80000.0,Primary Bank & Reserve,BM,1999,Road,21000,2,DSL,15
3,298,The Bike Mechanics,800000.0,80000.0,International Security,BM,1994,Mountain,18000,2,DSL,16
4,300,Nationwide Supply,800000.0,80000.0,Guardian Bank,BM,1987,Touring,21000,4+,DSL,17


In [11]:
#count the number of bike companies

pd.read_sql_query("""
SELECT COUNT(*)
  FROM AdventureWorks2019.Sales.vStoreWithDemographics;
""", connection)


,
0,701


In [12]:
#count the number of unique bike companies

pd.read_sql_query("""
SELECT COUNT(DISTINCT BusinessEntityID)
  FROM AdventureWorks2019.Sales.vStoreWithDemographics;
""", connection)


,
0,701


In [17]:
### check for the company specialty.  Mountain bike, Road bike, and Touring bikes


mountain = pd.read_sql_query("""
SELECT COUNT(Specialty)
  FROM AdventureWorks2019.Sales.vStoreWithDemographics
  WHERE Specialty = 'Mountain';
""", connection)

mountain.head()

,
0,246


In [18]:
##count how many companies specialize road v mountain v touring


road = pd.read_sql_query("""
SELECT COUNT(Specialty)
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Road';
""", connection)

road.head()

,
0,300


In [19]:
touring = pd.read_sql_query("""
SELECT COUNT(Specialty)
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Touring';
""", connection)

touring.head()

,
0,155


In [28]:
# i found the total sales and revenue of the companies that specialize in mountain, road and touring bikes
#road bikes were the best sellers

mountain_sales = pd.read_sql_query("""
SELECT SUM(AnnualSales) as total_sales, 
        SUM(AnnualRevenue) as total_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Mountain';
""", connection)

mountain_sales.head()

,total_sales,total_rev
0,405400000.0,40540000.0


In [29]:
road_sales = pd.read_sql_query("""
SELECT SUM(AnnualSales) as total_sales, 
        SUM(AnnualRevenue) as total_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Road';
""", connection)

road_sales.head()

,total_sales,total_rev
0,462400000.0,46240000.0


In [30]:
touring_sales = pd.read_sql_query("""
SELECT SUM(AnnualSales) as total_sales, 
        SUM(AnnualRevenue) as total_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Touring';
""", connection)

touring_sales.head()

,total_sales,total_rev
0,243100000.0,24310000.0


In [31]:
pd.read_sql_query("""
SELECT AVG(NumberEmployees)
FROM AdventureWorks2019.Sales.vStoreWithDemographics;
""", connection)

,
0,40


In [35]:
##does having bike or bicycle in company name increase sales

pd.read_sql_query("""
SELECT name
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE name LIKE '%bike%' or name LIKE '%bicycle%';
""", connection)

,name
0,Next-Door Bike Store
1,The Bike Mechanics
2,Area Bike Accessories
3,Bicycle Accessories and Kits
4,Valley Bicycle Specialists
...,...
236,Ultimate Bicycle Company
237,Solid Bike Parts
238,Great Bicycle Supply
239,World of Bikes


In [47]:
mtb = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Mountain'
and name LIKE '%bike%' or name LIKE '%cycle%';
""", connection)

mtb.head()

,avg_sales,avg_rev
0,1538418.079,153841.8079


In [48]:
mtb_no = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Mountain'
and name not LIKE '%bike%' or name not LIKE '%cycle%';
""", connection)

mtb_no.head()

,avg_sales,avg_rev
0,1602720.0,160272.0


In [49]:
rdb = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Road'
and name LIKE '%bike%' or name LIKE '%cycle%';
""", connection)

rdb.head()

,avg_sales,avg_rev
0,1.501546e+06,150154.6391


In [50]:
rd = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Road'
and name not LIKE '%bike%' or name not LIKE '%cycle%';
""", connection)

rd.head()

,avg_sales,avg_rev
0,1.594603e+06,159460.3174


In [51]:
t = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Touring'
and name LIKE '%bike%' or name LIKE '%cycle%';
""", connection)

t.head()

,avg_sales,avg_rev
0,1.511321e+06,151132.0754


In [52]:
tb = pd.read_sql_query("""
SELECT AVG(AnnualSales) as avg_sales, 
        AVG(AnnualRevenue) as avg_rev
FROM AdventureWorks2019.Sales.vStoreWithDemographics
WHERE Specialty = 'Touring'
and name not LIKE '%bike%' or name not LIKE '%cycle%';
""", connection)

tb.head()

,avg_sales,avg_rev
0,1.591558e+06,159155.8441
